In [6]:
library(dplyr)
library(lubridate)
library('ggplot2') 
library('ggthemes') 
library(magick)
library(grid)

ERROR: Error in library(magick): there is no package called ‘magick’


In [ ]:
# Load Arctic Sea Ice Extent data
# Source: ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/daily/data/N_seaice_extent_daily_v3.0.csv
# Downloaded on 22nd May 2020 (last day of the dataset)
seaice_N = read.csv("../data/N_seaice_extent_daily_v3.0.csv", stringsAsFactors=FALSE)

# Have a look at the table
head(seaice_N)  
tail(seaice_N)

In [ ]:
# Convert factors into integers
seaice_N$Year = as.integer(seaice_N$Year)
seaice_N$Month = as.integer(seaice_N$Month)
seaice_N$Day = as.integer(seaice_N$Day)

# Convert the Sea Ice Extent into numeric class
seaice_N$Extent = as.numeric(seaice_N$Extent)
seaice_N$Missing = as.numeric(seaice_N$Missing)

In [ ]:
# No missing values in the extent series
sum(is.na(seaice_N$Extent)) 

In [ ]:
seaice_N = seaice_N %>%
  select(-Source.Data, -Missing) %>% # Remove Source.Data and Missing columns
  mutate(Date = ymd(paste(Year, Month, Day, sep = "-"))) # Create Date column

ice = seaice_N %>%
  group_by(Year,Month) %>%
  summarize(avg_extent=mean(Extent)) 

In [ ]:
# Plotting all months doesn't provide any information
plot(1:500, ice$avg_extent, type = 'l', col='blue')

In [ ]:
# To make the annual mean, remove the incomplete years
# That's the months of 1978 and 2020
annual_mean = ice[-c(1, 2, 3, 496, 497, 498, 499, 500),]

annual_mean = annual_mean %>%
  group_by(Year) %>%
  summarize(annual_extent=mean(avg_extent))

In [ ]:
# Plot the annual mean time series
ggplot(annual_mean, aes(Year, annual_extent)) + labs(x='Years', y='Million sq Km') +
  geom_line(size=2, color = 'darkblue') + 
  labs(title="Annual Arctic Sea Ice Extent mean", fill="e")  +
  theme_gray(base_size = 18, base_family = "Times") +
  geom_smooth(method = "lm", se = FALSE, color='orange')

In [ ]:
# Annual Minimum Sea Ice Extent (September)
ice_september = ice[seq(12,500,12),]

In [ ]:
# Load the Sea Ice Concentration maps. Let's add them to the figure
# Source: "ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/images/09_Sep/N_201209_conc_blmrbl_v3.0.png"
sic_sep12 = image_read(paste("../data/N_201209_conc_blmrbl_v3.0.png", sep=''))
sic_sep80 = image_read(paste("../data/N_198009_conc_blmrbl_v3.0.png", sep=''))

# Transform into graphic object
g_12 =  rasterGrob(sic_sep12, interpolate=FALSE)
g_80 =  rasterGrob(sic_sep80, interpolate=FALSE)

In [ ]:
# The plot
ggplot(ice_september, aes(Year, avg_extent)) + labs(x='Years', y='Million sq Km') +
  geom_line(size=2, color = 'darkgreen') + 
  labs(title="September Arctic Sea Ice Extent", fill="e")  +
  theme_gray(base_size = 18, base_family = "Times") +
  geom_smooth(method = "lm", se = FALSE, color='orange') +
  annotation_custom(grob=g_12, xmin=1986, xmax=2006, ymin=2, ymax=5.5) +
  annotation_custom(grob=g_80, xmin=1971, xmax=1991, ymin=2, ymax=5.5) +
  geom_point(aes(x=1980, y=as.numeric(ice_september[2,3])), colour="green3",
             fill="green3", shape=21, size=5) +
  geom_point(aes(x=2012, y=as.numeric(ice_september[34,3])), colour="red3",
             fill="red3", shape=21, size=5) +
  annotate(geom="text", x=1982, y=5.5, label="1980",
           color="green3", size = 5) +
  annotate(geom="text", x=1997, y=5.5, label="2012",
           color="red3", size = 5, font=2) 